In [ ]:
import time
import transformer_ad
import pandas as pd
import pickle as pk
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import itertools
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as prf

import random
import torch
import os
import gc

In [ ]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
#tf.set_random_seed(seed)

In [ ]:
def shift(arr, num, fill_value=np.nan):
    arr = np.roll(arr, num)
    if num < 0:
        arr[num:] = fill_value
    elif num > 0:
        arr[:num] = fill_value
    return arr
def filter_pred(values_pred, scale=3):
    predicted_anomalies_ = np.argwhere(values_pred == 1).ravel()
    predicted_anomalies_shift_forward = shift(predicted_anomalies_, 1, fill_value=predicted_anomalies_[0])
    predicted_anomalies_shift_backward = shift(predicted_anomalies_, -1, fill_value=predicted_anomalies_[-1])
    predicted_anomalies_start = np.argwhere(
        (predicted_anomalies_shift_forward - predicted_anomalies_) != -1
    ).ravel()
    predicted_anomalies_finish = np.argwhere(
        (predicted_anomalies_ - predicted_anomalies_shift_backward) != -1
    ).ravel()
    predicted_anomalies = np.hstack(
        [
            predicted_anomalies_[predicted_anomalies_start].reshape(-1, 1),
            predicted_anomalies_[predicted_anomalies_finish].reshape(-1, 1),
        ]
    )
    for a_range in predicted_anomalies:
        if a_range[1]-a_range[0]<=scale-1:
            values_pred[a_range[0]:a_range[1]+1] = 0
    return values_pred

from prts import ts_precision, ts_recall, ts_fscore
def bf_search(label, score, verbose=True, is_filter=False):
    """
    Find the best-f1 score by searching best `threshold` in [`start`, `end`).
    Returns:
        list: list for results
        float: the `threshold` for best-f1
    """
    start = 90
    search_range = [np.percentile(score, q) for q in np.arange(start, 100, 0.1)]
    m = {'f1-score':-1., 'precision':-1., 'recall':-1.}
    m_t = 0.0
    #print(len(score))
    #print(len(search_range))
    for threshold in sorted(search_range)[::-1]:
        real = label
        pred = score > threshold
        #print(np.unique(pred))
        if is_filter:
            pred = filter_pred(pred, scale=1)
        #pred = filter_pred(pred, scale=3)
        if len(np.unique(pred))==1:
            continue
        target = ts_fscore(real, pred, beta=1.0, p_alpha=0.0, r_alpha=0.5, cardinality="reciprocal", p_bias="front", r_bias="front")
        if target > m['f1-score']:
            m_t = threshold
            m['f1-score'] = target
            m['precision'] = ts_precision(real, pred, alpha=0.0, cardinality="reciprocal", bias="front")
            m['recall'] = ts_recall(real, pred, alpha=0.5, cardinality="reciprocal", bias="front")
            if verbose:
                print("cur thr: ", threshold, target, m, m_t)
    #print(m, m_t)
    return m, m_t

def calc_point2point(predict, actual):
    """
    calculate f1 score by predict and actual.

    Args:
        predict (np.ndarray): the predict label
        actual (np.ndarray): np.ndarray
    """
    TP = np.sum(predict * actual)
    TN = np.sum((1 - predict) * (1 - actual))
    FP = np.sum(predict * (1 - actual))
    FN = np.sum((1 - predict) * actual)
    precision = TP / (TP + FP + 0.00001)
    recall = TP / (TP + FN + 0.00001)
    f1 = 2 * precision * recall / (precision + recall + 0.00001)
    return f1, precision, recall

def adjust_predicts(score, label,
                    threshold=None,
                    pred=None,
                    calc_latency=False,
                    is_filter=False):
    """
    Calculate adjusted predict labels using given `score`, `threshold` (or given `pred`) and `label`.

    Args:
        score (np.ndarray): The anomaly score
        label (np.ndarray): The ground-truth label
        threshold (float): The threshold of anomaly score.
            A point is labeled as "anomaly" if its score is lower than the threshold.
        pred (np.ndarray or None): if not None, adjust `pred` and ignore `score` and `threshold`,
        calc_latency (bool):

    Returns:
        np.ndarray: predict labels
    """
    if len(score) != len(label):
        raise ValueError("score and label must have the same length")
    score = np.asarray(score)
    label = np.asarray(label)
    latency = 0
    if pred is None:
        predict = score > threshold
        if is_filter:
            predict = filter_pred(predict, scale=1)
    else:
        predict = pred
    actual = label > 0.1
    anomaly_state = False
    anomaly_count = 0
    for i in range(len(score)):
        if actual[i] and predict[i] and not anomaly_state:
            anomaly_state = True
            anomaly_count += 1
            for j in range(i, 0, -1):
                if not actual[j]:
                    break
                else:
                    if not predict[j]:
                        predict[j] = True
                        latency += 1
        elif not actual[i]:
            anomaly_state = False
        if anomaly_state:
            predict[i] = True
    if calc_latency:
        return predict, latency / (anomaly_count + 1e-4)
    else:
        return predict

def bf_search_omni(label, score, verbose=True, is_filter=False):
    """
    Find the best-f1 score by searching best `threshold` in [`start`, `end`).
    Returns:
        list: list for results
        float: the `threshold` for best-f1
    """
    start = 90
    search_range = [np.percentile(score, q) for q in np.arange(start, 100, 0.1)]
    m = {'f1-score':-1., 'precision':-1., 'recall':-1.}
    m_t = 0.0
    #print(len(score))
    #print(len(search_range))
    for threshold in sorted(search_range)[::-1]:
        real = label
        pred = adjust_predicts(score, label, threshold, is_filter=is_filter)
        #print(np.unique(pred))
        if len(np.unique(pred))==1:
            continue
        target = calc_point2point(pred, label)
        if target[0] > m['f1-score']:
            m_t = threshold
            m['f1-score'] = target[0]
            m['precision'] = target[1]
            m['recall'] = target[2]
            if verbose:
                print("cur thr: ", threshold, target, m, m_t)
    #print(m, m_t)
    return m, m_t

In [ ]:
def ap_rangebased(label, score, is_filter=False):
    """
    Find the best-f1 score by searching best `threshold` in [`start`, `end`).
    Returns:
        list: list for results
        float: the `threshold` for best-f1
    """
    start = 0
    search_range = [np.percentile(score, q) for q in np.arange(start, 100, 0.1)]
    m = {}
    m['precision'] = []
    m['recall'] = []
    for threshold in sorted(search_range):
        real = label
        pred = score > threshold
        #print(np.unique(pred))
        if is_filter:
            pred = filter_pred(pred, scale=1)
        #pred = filter_pred(pred, scale=3)
        if len(np.unique(pred))==1:
            continue
        m['precision'].append(ts_precision(real, pred, alpha=0.0, cardinality="reciprocal", bias="front"))
        m['recall'].append(ts_recall(real, pred, alpha=0.5, cardinality="reciprocal", bias="front"))
    # The last precision and recall values are 1. and 0
    m['precision'].append(1)
    m['recall'].append(0)
    ap = -np.sum(np.diff(m['recall']) * np.array(m['precision'])[1:])
    #print(m, ap)
    #print(ap)
    return ap

def ap_pointadjust(label, score, is_filter=False):
    """
    Find the best-f1 score by searching best `threshold` in [`start`, `end`).
    Returns:
        list: list for results
        float: the `threshold` for best-f1
    """
    start = 0
    search_range = [np.percentile(score, q) for q in np.arange(start, 100, 0.1)]
    m = {}
    m['precision'] = []
    m['recall'] = []
    for threshold in sorted(search_range):
        real = label
        pred = adjust_predicts(score, label, threshold, is_filter=is_filter)
        #print(np.unique(pred))
        if len(np.unique(pred))==1:
            continue
        target = calc_point2point(pred, label)
        m['precision'].append(target[1])
        m['recall'].append(target[2])
    # The last precision and recall values are 1. and 0
    m['precision'].append(1)
    m['recall'].append(0)
    ap = -np.sum(np.diff(m['recall']) * np.array(m['precision'])[1:])
    #print(m, ap)
    #print(ap)
    return ap

In [ ]:
datafiles = sorted(os.listdir('CTF_data/CTF_data'))

In [ ]:
datafiles.pop(0)

In [ ]:
datafiles = [f for f in datafiles if '118' not in f and '166' not in f and '174' not in f and '263' not in f and '289' not in f and '307' not in f and '419' not in f and '432' not in f]

In [ ]:
len(datafiles)

In [ ]:
len(datafiles)/13

In [ ]:
#name='iterate-huber-nosfa-nolongterm-ep30-1e-4-noisy'
lr=1e-4
num_epochs=5
sequence_length = 100
z_dim = 16
batch_size = 128
gpu_choice = 3
use_sfa = False
no_longterm = True
no_featerm = False
noisy_rate = 0
loss_func = 'huber'


In [ ]:
name = 'transformer_ad'
name += '-'
name += loss_func
name += '-'
if use_sfa:
    name += 'usesfa'
    name += '-'
else:
    name += 'nosfa'
    name += '-'
if no_longterm:
    name += 'nolongterm'
    name += '-'
else:
    name += 'uselongterm'
    name += '-'
if no_featerm:
    name += 'nofeaterm'
    name += '-'
else:
    name += 'usefeaterm'
    name += '-'
name += 'ep'
name += str(num_epochs)
name += '-'
name += 'z'
name += str(z_dim)
name += '-'
if noisy_rate != 0:
    name += 'noisy'
    name += str(noisy_rate)
    name += '-'
else:
    name += 'nonoisy'
basename = name
name += time.strftime("%Y-%m-%d-%H:%M:%S", time.localtime()) 
print(basename)

In [ ]:
%%time
dataset = transformer_ad.MonitorEntityDataset(datafiles, sequence_length, z_dim, gpu=gpu_choice, use_sfa=use_sfa, no_longterm=no_longterm)

In [ ]:
lr_range = [1e-4, 1e-3]
hid_range = [3, 8, 16]

point_adjust_f1_LIST = []
range_based_f1_LIST = []

point_adjust_ap_LIST = []
range_based_ap_LIST = []

#with open(basename+'_transformerad_experiment_results.txt', 'a') as f:
#    f.write(name+'\n')
with open('transformerad_experiment_results_CTFDATA.txt', 'a') as f:
    f.write(name+'\n')
for lr_example in lr_range:
    for hid_example in hid_range:
        name = 'transformer_ad'
        name += '-'
        name += loss_func
        name += '-'
        if use_sfa:
            name += 'usesfa'
            name += '-'
        else:
            name += 'nosfa'
            name += '-'
        if no_longterm:
            name += 'nolongterm'
            name += '-'
        else:
            name += 'uselongterm'
            name += '-'
        if no_featerm:
            name += 'nofeaterm'
            name += '-'
        else:
            name += 'usefeaterm'
            name += '-'
        name += 'ep'
        name += str(num_epochs)
        name += '-'
        name += 'z'
        name += '-'
        if noisy_rate != 0:
            name += 'noisy'
            name += str(noisy_rate)
            name += '-'
        else:
            name += 'nonoisy'
        basename = name
        name += time.strftime("%Y-%m-%d-%H:%M:%S", time.localtime()) 
        print(name)
        
        
        dataset.train()
        model = transformer_ad.Transformer_AD(name=name, lr=lr_example, num_epochs=num_epochs, hidden_dim=hid_example, batch_size=batch_size, gpu=gpu_choice, use_sfa=use_sfa, no_longterm=no_longterm, no_featerm=no_featerm, noisy_rate=noisy_rate)
        model.fit(dataset, loss_func=loss_func, log_step=60)
        
        mac_ids = sorted(list(set([f.split('_')[0] for f in datafiles])))
        aucs = []
        prfs = {}
        prfs_omni = {}
        
        aps_rbased = {}
        aps_padjust = {}
        gc.collect()
        for i in range(len(mac_ids)):
            print(mac_ids[i])

            dataset.test(mac_ids[i])
            with open('CTF_data/label_result/' + mac_ids[i]+'.pkl', 'rb') as label_file:
                label = pk.load(label_file)

            pred = model.predict(dataset)
            pred = np.mean(pred, axis = 2)[1:,-1]

            aucs.append(roc_auc_score(label, pred))
    
            prfs[mac_ids[i]] = bf_search(label.flatten(), pred, verbose=False)

            prfs_omni[mac_ids[i]] = bf_search_omni(label.flatten(), pred, verbose=False)
            
            aps_rbased[mac_ids[i]] = ap_rangebased(label, pred)
            aps_padjust[mac_ids[i]] = ap_pointadjust(label, pred)
            
        omnifscore = [item[0]['f1-score'] for item in list(prfs_omni.values())]
        point_adjust_f1 = np.mean(omnifscore)

        rangefscore = [item[0]['f1-score'] for item in list(prfs.values())]
        range_based_f1 = np.mean(rangefscore)
        
        point_adjust_ap = np.mean(list(aps_padjust.values()))
        range_based_ap = np.mean(list(aps_rbased.values()))
        
        point_adjust_f1_LIST.append(point_adjust_f1)
        range_based_f1_LIST.append(range_based_f1)

        point_adjust_ap_LIST.append(point_adjust_ap)
        range_based_ap_LIST.append(range_based_ap)

        with open('transformerad_experiment_results_CTFDATA.txt', 'a') as f:
            f.write('lr: '+str(lr_example)+', hid: '+str(hid_example)+', point_adjust_f1: '+str(point_adjust_f1)+', range_based_f1: '+str(range_based_f1)+', point_adjust_ap: '+str(point_adjust_ap)+', range_based_ap: '+str(range_based_ap)+'\n')
            
with open('transformerad_experiment_results_CTFDATA.txt', 'a') as f:
    f.write(name+'\n')

In [ ]:
print('point_adjust_f1_LIST =', point_adjust_f1_LIST)
print('range_based_f1_LIST =', range_based_f1_LIST)

print('point_adjust_ap_LIST =', point_adjust_ap_LIST)
print('range_based_ap_LIST =', range_based_ap_LIST)